In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import parallel_coordinates
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)
pd.options.display.max_colwidth = 30

/Users/leejaeuk/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


# 1. chain 데이터 

In [3]:
chain = pd.read_csv('./chain_data.csv').drop(columns='Unnamed: 0')
chain

,chain,x-axis,y-axis,radius
0,cosmos,-3.932109,-3.136361,16.389515
1,kava,-0.472359,1.235081,14.392602
2,injective,-2.156337,-5.232898,14.819475
3,axelar,-3.018877,-1.306728,13.776378
4,osmosis,-5.526520,-1.757991,16.635813
5,terra,-0.076367,-7.691177,17.385445
6,iris,3.950026,2.738528,16.020174
7,dydx,-2.093366,-1.113769,14.363712
8,secret,4.117796,-3.661395,15.788137
9,akash,-0.494838,0.632218,12.711289


In [4]:
chain_list = list(chain['chain'])
chain_list

['cosmos',
 'kava',
 'injective',
 'axelar',
 'osmosis',
 'terra',
 'iris',
 'dydx',
 'secret',
 'akash',
 'stargaze',
 'gravity-bridge',
 'sentinel',
 'juno']

# 2. 월별 proposal 수

In [5]:
import os
import pandas as pd
import numpy as np

folder_path = './new_data/'

all_data = {}

# 각 파일마다 데이터를 불러와서 전처리 후 리스트에 추가
for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):
        chain_name = file_name.replace('_proposals', '').replace('.csv', '')
        df = pd.read_csv(folder_path + file_name).drop(columns=['Unnamed: 0'])

        if chain_name not in chain_list:
            continue
        
        print(chain_name)

        # 불필요한 열 제거
        df = df.drop(columns=['txHash', 'txHeight', 'timeSubmit', 'proposer'], errors='ignore')
        df['timeVotingStart'] = pd.to_datetime(df['timeVotingStart'], unit='ms')
        df['timeVotingEnd'] = pd.to_datetime(df['timeVotingEnd'], unit='ms')
        
        # Filter rows for timeVotingEnd within December 2023 to November 2024
        df = df[(df['timeVotingEnd'] >= '2023-12-01') & (df['timeVotingEnd'] < '2024-12-01')]
        
        def get_quarter_group(date):
            if date.month in [12, 1, 2]:
                return '23.12-24.02'
            elif date.month in [3, 4, 5]:
                return '24.03-24.05'
            elif date.month in [6, 7, 8]:
                return '24.06-24.08'
            elif date.month in [9, 10, 11]:
                return '24.09-24.11'
        
        df['quarter_group'] = df['timeVotingEnd'].apply(get_quarter_group)

        # Count proposals by quarter_group
        quarter_counts = df['quarter_group'].value_counts().sort_index()
        quarter_df = quarter_counts.reset_index()
        quarter_df.columns = ['quarter_group', 'count']
        
        # Add the quarter data to the chain DataFrame
        dictQ = dict(zip(quarter_df['quarter_group'], quarter_df['count']))
        chain.loc[chain['chain'] == chain_name, 'proposal'] = [dictQ]

chain

cosmos
kava
injective
axelar
osmosis
terra
iris
dydx
secret
akash
stargaze
gravity-bridge
sentinel
juno


,chain,x-axis,y-axis,radius,proposal
0,cosmos,-3.932109,-3.136361,16.389515,"{'23.12-24.02': 25, '24.03..."
1,kava,-0.472359,1.235081,14.392602,"{'23.12-24.02': 7, '24.03-..."
2,injective,-2.156337,-5.232898,14.819475,"{'23.12-24.02': 49, '24.03..."
3,axelar,-3.018877,-1.306728,13.776378,"{'23.12-24.02': 5, '24.03-..."
4,osmosis,-5.526520,-1.757991,16.635813,"{'23.12-24.02': 50, '24.03..."
5,terra,-0.076367,-7.691177,17.385445,"{'23.12-24.02': 6, '24.03-..."
6,iris,3.950026,2.738528,16.020174,"{'23.12-24.02': 20, '24.03..."
7,dydx,-2.093366,-1.113769,14.363712,"{'23.12-24.02': 24, '24.03..."
8,secret,4.117796,-3.661395,15.788137,"{'23.12-24.02': 8, '24.03-..."
9,akash,-0.494838,0.632218,12.711289,"{'23.12-24.02': 9, '24.03-..."


In [6]:
chain['proposal'][2]

{'23.12-24.02': 49, '24.03-24.05': 49, '24.06-24.08': 27, '24.09-24.11': 30}

# 3. 체인 별 cluster 비율

In [7]:
votes = pd.read_csv('./complete_data/validator_votes.csv').drop(columns='Unnamed: 0')
votes

/var/folders/nc/dhlk15l910g4hrbcq92y1cy40000gn/T/ipykernel_3603/3495828067.py:1: DtypeWarning: Columns (2,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,61,63,64,65,67,68,69,70,71,72,73,74,75,76,77,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,

,voter,osmosis_1,osmosis,osmosis_3,osmosis_5,osmosis_7,osmosis_22,osmosis_24,osmosis_29,osmosis_39,osmosis_50,osmosis_51,osmosis_55,osmosis_57,osmosis_58,osmosis_62,osmosis_72,osmosis_74,osmosis_76,osmosis_81,osmosis_82,osmosis_84,osmosis_96,osmosis_109,osmosis_111,osmosis_114,osmosis_120,osmosis_123,osmosis_130,osmosis_132,osmosis_133,osmosis_134,osmosis_135,osmosis_140,osmosis_141,osmosis_144,osmosis_146,osmosis_148,osmosis_149,osmosis_151,osmosis_152,osmosis_153,osmosis_156,osmosis_159,osmosis_163,osmosis_165,osmosis_166,osmosis_169,osmosis_170,osmosis_171,osmosis_173,osmosis_178,osmosis_180,osmosis_188,osmosis_190,osmosis_191,osmosis_211,osmosis_306,juno_4,juno,stargaze_11,stargaze,stargaze_15,juno_16,akash_16,akash,stargaze_16,stargaze_17,juno_18,juno_19,juno_20,stargaze_50,akash_52,stargaze_57,stargaze_58,stargaze_59,cosmos_60,cosmos,cosmos_63,cosmos_69,juno_72,cosmos_72,cosmos_75,cosmos_76,cosmos_77,cosmos_78,cosmos_79,cosmos_80,cosmos_81,stargaze_81,cosmos_82,juno_83,cosmos_83,juno_85,cosmos_86,cosmos_89,cosmos_90,cosmos_93,cosmos_94,stargaze_94,cosmos_99,cosmos_101,stargaze_104,juno_125,juno_141,osmosis_162,akash_182,akash_185,akash_187,osmosis_194,osmosis_195,osmosis_199,osmosis_200,osmosis_204,osmosis_206,osmosis_207,osmosis_208,osmosis_209,osmosis_216,osmosis_220,akash_229,akash_232,osmosis_232,akash_233,akash_234,akash_236,osmosis_237,akash_242,akash_243,akash_244,akash_245,akash_246,akash_247,akash_250,osmosis_251,akash_252,akash_253,akash_254,akash_255,osmosis_256,akash_261,osmosis_261,osmosis_262,akash_263,akash_266,osmosis_267,akash_268,osmosis_270,osmosis_271,osmosis_273,juno_284,juno_292,osmosis_305,juno_307,osmosis_309,osmosis_315,osmosis_316,osmosis_323,osmosis_326,osmosis_339,osmosis_342,osmosis_343,osmosis_349,osmosis_350,osmosis_362,osmosis_365,osmosis_367,osmosis_372,osmosis_378,osmosis_386,osmosis_388,osmosis_389,osmosis_390,osmosis_400,osmosis_402,osmosis_403,osmosis_419,osmosis_420,osmosis_421,osmosis_436,osmosis_442,osmosis_444,osmosis_451,osmosis_457,osmosis_480,osmosis_482,osmosis_492,osmosis_497,osmosis_498,osmosis_519,osmosis_530,osmosis_561,osmosis_564,osmosis_593,osmosis_610,osmosis_620,osmosis_638,osmosis_639,osmosis_640,osmosis_641,osmosis_645,osmosis_652,osmosis_655,cosmos_687,osmosis_688,osmosis_694,osmosis_700,osmosis_706,osmosis_707,osmosis_712,osmosis_741,osmosis_742,osmosis_755,osmosis_764,osmosis_785,cosmos_787,cosmos_791,cosmos_793,cosmos_800,cosmos_811,cosmos_814,cosmos_818,cosmos_826,cosmos_827,cosmos_836,cosmos_839,cosmos_848,cosmos_851,cosmos_853,cosmos_856,cosmos_858,cosmos_861,cosmos_862,cosmos_865,cosmos_867,cosmos_868,cosmos_895,cosmos_899,cosmos_912,cosmos_916,cosmos_917,cosmos_946,cosmos_952,cosmos_954,cosmos_956,osmosis_276,osmosis_277,osmosis_320,osmosis_352,osmosis_376,osmosis_394,osmosis_406,osmosis_417,osmosis_464,dydx_6,dydx,juno_7,akash_9,akash_14,juno_15,akash_22,akash_23,akash_24,iris_24,iris,dydx_24,iris_25,iris_26,akash_26,akash_28,iris_28,axelar_29,axelar,juno_30,akash_31,stargaze_35,juno_39,iris_41,dydx_44,iris_47,dydx_47,iris_48,dydx_48,iris_52,juno_59,juno_60,akash_63,cosmos_68,cosmos_73,iris_73,iris_74,cosmos_74,iris_76,iris_77,iris_78,iris_79,iris_84,iris_89,juno_89,cosmos_95,cosmos_98,stargaze_101,cosmos_103,cosmos_104,stargaze_128,dydx_138,dydx_142,stargaze_144,dydx_144,dydx_154,cosmos_155,dydx_156,dydx_157,dydx_164,dydx_165,axelar_166,axelar_167,axelar_168,axelar_169,axelar_170,axelar_171,axelar_172,injective_173,injective,axelar_174,dydx_175,dydx_176,axelar_178,axelar_188,axelar_190,axelar_191,axelar_192,axelar_193,injective_193,axelar_194,axelar_196,akash_199,axelar_199,akash_200,stargaze_200,axelar_200,axelar_201,cosmos_202,axelar_204,stargaze_205,axelar_206,akash_206,axelar_207,akash_207,akash_208,axelar_208,axelar_209,stargaze_210,axelar_211,axelar_212,axelar_213,stargaze_213,axelar_215,akash_216,axelar_216,axelar_217,axelar_218,axelar_219,axelar_220,axelar_222,axelar_223,stargaze_223,injective_226,stargaze_226,akash_230,akash_235,axelar_239,axelar

In [8]:
filter_col = ['voter'] + chain_list + ['cluster_label']
tvotes = votes[[col for col in votes.columns if col in filter_col]]
tvotes

,voter,osmosis,juno,stargaze,akash,cosmos,dydx,iris,axelar,injective,terra,sentinel,secret,kava,gravity-bridge,cluster_label
0,Inactive Mercury,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
1,Chainflow,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
2,REDELEGATE CLOSING 010723,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
3,Imperatorco,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,3
4,AutoStake Slash Protected,1.0,1.0,1.0,1.0,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,NaN,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1228,Midora,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2
1229,Eclipse Nodes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2
1230,PowerStaking,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2
1231,Ansybl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2


In [9]:
cluster_counts = tvotes.groupby('cluster_label')[chain_list].apply(lambda x: x.notna().sum())

# Reset index to convert it into a clean DataFrame
cluster_counts = cluster_counts.reset_index()
cluster_counts

,cluster_label,cosmos,kava,injective,axelar,osmosis,terra,iris,dydx,secret,akash,stargaze,gravity-bridge,sentinel,juno
0,0,5,2,4,1,5,20,3,5,71,5,6,1,34,3
1,1,15,8,3,1,23,6,5,6,2,14,90,16,12,107
2,2,35,19,2,33,52,13,14,6,15,22,18,41,7,17
3,3,70,23,28,24,116,19,10,25,15,37,44,18,23,60
4,4,7,4,2,1,3,1,67,19,3,38,0,6,7,1
5,5,5,50,1,4,5,1,4,1,4,17,8,98,17,11
6,6,16,8,55,10,10,85,7,10,5,6,7,6,2,4
7,7,128,11,7,9,45,9,11,11,4,16,9,12,2,23


In [10]:
normalized_counts = cluster_counts.copy()
columns_to_normalize = normalized_counts.columns[1:]  # Exclude 'cluster_label'

for col in columns_to_normalize:
    total_sum = normalized_counts[col].sum()
    normalized_counts[col] = normalized_counts[col] / total_sum
normalized_counts

,cluster_label,cosmos,kava,injective,axelar,osmosis,terra,iris,dydx,secret,akash,stargaze,gravity-bridge,sentinel,juno
0,0,0.017794,0.016,0.039216,0.012048,0.019305,0.129870,0.024793,0.060241,0.596639,0.032258,0.032967,0.005051,0.326923,0.013274
1,1,0.053381,0.064,0.029412,0.012048,0.088803,0.038961,0.041322,0.072289,0.016807,0.090323,0.494505,0.080808,0.115385,0.473451
2,2,0.124555,0.152,0.019608,0.397590,0.200772,0.084416,0.115702,0.072289,0.126050,0.141935,0.098901,0.207071,0.067308,0.075221
3,3,0.249110,0.184,0.274510,0.289157,0.447876,0.123377,0.082645,0.301205,0.126050,0.238710,0.241758,0.090909,0.221154,0.265487
4,4,0.024911,0.032,0.019608,0.012048,0.011583,0.006494,0.553719,0.228916,0.025210,0.245161,0.000000,0.030303,0.067308,0.004425
5,5,0.017794,0.400,0.009804,0.048193,0.019305,0.006494,0.033058,0.012048,0.033613,0.109677,0.043956,0.494949,0.163462,0.048673
6,6,0.056940,0.064,0.539216,0.120482,0.038610,0.551948,0.057851,0.120482,0.042017,0.038710,0.038462,0.030303,0.019231,0.017699
7,7,0.455516,0.088,0.068627,0.108434,0.173745,0.058442,0.090909,0.132530,0.033613,0.103226,0.049451,0.060606,0.019231,0.101770


In [11]:
proportions_dict = {
    chain: normalized_counts.set_index('cluster_label')[chain].to_dict()
    for chain in chain_list
}

# Add proportions to the second DataFrame
chain['proportion'] = chain['chain'].map(proportions_dict)
chain

,chain,x-axis,y-axis,radius,proposal,proportion
0,cosmos,-3.932109,-3.136361,16.389515,"{'23.12-24.02': 25, '24.03...","{0: 0.017793594306049824, ..."
1,kava,-0.472359,1.235081,14.392602,"{'23.12-24.02': 7, '24.03-...","{0: 0.016, 1: 0.064, 2: 0...."
2,injective,-2.156337,-5.232898,14.819475,"{'23.12-24.02': 49, '24.03...","{0: 0.0392156862745098, 1:..."
3,axelar,-3.018877,-1.306728,13.776378,"{'23.12-24.02': 5, '24.03-...","{0: 0.012048192771084338, ..."
4,osmosis,-5.526520,-1.757991,16.635813,"{'23.12-24.02': 50, '24.03...","{0: 0.019305019305019305, ..."
5,terra,-0.076367,-7.691177,17.385445,"{'23.12-24.02': 6, '24.03-...","{0: 0.12987012987012986, 1..."
6,iris,3.950026,2.738528,16.020174,"{'23.12-24.02': 20, '24.03...","{0: 0.024793388429752067, ..."
7,dydx,-2.093366,-1.113769,14.363712,"{'23.12-24.02': 24, '24.03...","{0: 0.060240963855421686, ..."
8,secret,4.117796,-3.661395,15.788137,"{'23.12-24.02': 8, '24.03-...","{0: 0.5966386554621849, 1:..."
9,akash,-0.494838,0.632218,12.711289,"{'23.12-24.02': 9, '24.03-...","{0: 0.03225806451612903, 1..."


# 4. 체인 별 validator, proposal, cluster 개수

In [12]:
validator_counts = cluster_counts.drop(columns=['cluster_label'])
for i in range(len(validator_counts.sum())):
    chain.loc[i, 'validator_num'] = validator_counts.sum()[i]

chain

/var/folders/nc/dhlk15l910g4hrbcq92y1cy40000gn/T/ipykernel_3603/2568880261.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  chain.loc[i, 'validator_num'] = validator_counts.sum()[i]
/var/folders/nc/dhlk15l910g4hrbcq92y1cy40000gn/T/ipykernel_3603/2568880261.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  chain.loc[i, 'validator_num'] = validator_counts.sum()[i]
/var/folders/nc/dhlk15l910g4hrbcq92y1cy40000gn/T/ipykernel_3603/2568880261.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame beh

,chain,x-axis,y-axis,radius,proposal,proportion,validator_num
0,cosmos,-3.932109,-3.136361,16.389515,"{'23.12-24.02': 25, '24.03...","{0: 0.017793594306049824, ...",281.0
1,kava,-0.472359,1.235081,14.392602,"{'23.12-24.02': 7, '24.03-...","{0: 0.016, 1: 0.064, 2: 0....",125.0
2,injective,-2.156337,-5.232898,14.819475,"{'23.12-24.02': 49, '24.03...","{0: 0.0392156862745098, 1:...",102.0
3,axelar,-3.018877,-1.306728,13.776378,"{'23.12-24.02': 5, '24.03-...","{0: 0.012048192771084338, ...",83.0
4,osmosis,-5.526520,-1.757991,16.635813,"{'23.12-24.02': 50, '24.03...","{0: 0.019305019305019305, ...",259.0
5,terra,-0.076367,-7.691177,17.385445,"{'23.12-24.02': 6, '24.03-...","{0: 0.12987012987012986, 1...",154.0
6,iris,3.950026,2.738528,16.020174,"{'23.12-24.02': 20, '24.03...","{0: 0.024793388429752067, ...",121.0
7,dydx,-2.093366,-1.113769,14.363712,"{'23.12-24.02': 24, '24.03...","{0: 0.060240963855421686, ...",83.0
8,secret,4.117796,-3.661395,15.788137,"{'23.12-24.02': 8, '24.03-...","{0: 0.5966386554621849, 1:...",119.0
9,akash,-0.494838,0.632218,12.711289,"{'23.12-24.02': 9, '24.03-...","{0: 0.03225806451612903, 1...",155.0


In [13]:
import os
import json
import pandas as pd
import numpy as np

folder_path = './new_data/'
chain_list = [
    'cosmos', 'kava', 'injective', 'axelar', 'osmosis', 'terra',
    'iris', 'dydx', 'secret', 'akash', 'stargaze', 'gravity-bridge',
    'sentinel', 'juno'
]

# 모든 파일 불러오기 및 병합
proposal_df = []
i = 0
# 각 파일마다 데이터를 불러와서 전처리 후 리스트에 추가
for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):
        chain_name = file_name.replace('_proposals', '').replace('.csv', '')
        df = pd.read_csv(folder_path + file_name).drop(columns=['Unnamed: 0'])
        
        if chain_name in chain_list:
            chain.loc[i, 'proposal_num'] = len(df)
            i+=1

chain

,chain,x-axis,y-axis,radius,proposal,proportion,validator_num,proposal_num
0,cosmos,-3.932109,-3.136361,16.389515,"{'23.12-24.02': 25, '24.03...","{0: 0.017793594306049824, ...",281.0,249.0
1,kava,-0.472359,1.235081,14.392602,"{'23.12-24.02': 7, '24.03-...","{0: 0.016, 1: 0.064, 2: 0....",125.0,188.0
2,injective,-2.156337,-5.232898,14.819475,"{'23.12-24.02': 49, '24.03...","{0: 0.0392156862745098, 1:...",102.0,406.0
3,axelar,-3.018877,-1.306728,13.776378,"{'23.12-24.02': 5, '24.03-...","{0: 0.012048192771084338, ...",83.0,123.0
4,osmosis,-5.526520,-1.757991,16.635813,"{'23.12-24.02': 50, '24.03...","{0: 0.019305019305019305, ...",259.0,791.0
5,terra,-0.076367,-7.691177,17.385445,"{'23.12-24.02': 6, '24.03-...","{0: 0.12987012987012986, 1...",154.0,103.0
6,iris,3.950026,2.738528,16.020174,"{'23.12-24.02': 20, '24.03...","{0: 0.024793388429752067, ...",121.0,85.0
7,dydx,-2.093366,-1.113769,14.363712,"{'23.12-24.02': 24, '24.03...","{0: 0.060240963855421686, ...",83.0,175.0
8,secret,4.117796,-3.661395,15.788137,"{'23.12-24.02': 8, '24.03-...","{0: 0.5966386554621849, 1:...",119.0,229.0
9,akash,-0.494838,0.632218,12.711289,"{'23.12-24.02': 9, '24.03-...","{0: 0.03225806451612903, 1...",155.0,95.0


In [14]:
for i in range(len(chain)):
    non_zero_keys_count = len([key for key, value in chain['proportion'][i].items() if value != 0])
    chain.loc[i, 'cluster_num'] = non_zero_keys_count

chain

,chain,x-axis,y-axis,radius,proposal,proportion,validator_num,proposal_num,cluster_num
0,cosmos,-3.932109,-3.136361,16.389515,"{'23.12-24.02': 25, '24.03...","{0: 0.017793594306049824, ...",281.0,249.0,8.0
1,kava,-0.472359,1.235081,14.392602,"{'23.12-24.02': 7, '24.03-...","{0: 0.016, 1: 0.064, 2: 0....",125.0,188.0,8.0
2,injective,-2.156337,-5.232898,14.819475,"{'23.12-24.02': 49, '24.03...","{0: 0.0392156862745098, 1:...",102.0,406.0,8.0
3,axelar,-3.018877,-1.306728,13.776378,"{'23.12-24.02': 5, '24.03-...","{0: 0.012048192771084338, ...",83.0,123.0,8.0
4,osmosis,-5.526520,-1.757991,16.635813,"{'23.12-24.02': 50, '24.03...","{0: 0.019305019305019305, ...",259.0,791.0,8.0
5,terra,-0.076367,-7.691177,17.385445,"{'23.12-24.02': 6, '24.03-...","{0: 0.12987012987012986, 1...",154.0,103.0,8.0
6,iris,3.950026,2.738528,16.020174,"{'23.12-24.02': 20, '24.03...","{0: 0.024793388429752067, ...",121.0,85.0,8.0
7,dydx,-2.093366,-1.113769,14.363712,"{'23.12-24.02': 24, '24.03...","{0: 0.060240963855421686, ...",83.0,175.0,8.0
8,secret,4.117796,-3.661395,15.788137,"{'23.12-24.02': 8, '24.03-...","{0: 0.5966386554621849, 1:...",119.0,229.0,8.0
9,akash,-0.494838,0.632218,12.711289,"{'23.12-24.02': 9, '24.03-...","{0: 0.03225806451612903, 1...",155.0,95.0,8.0


# 5. 유사한 체인

In [15]:
from sklearn.metrics.pairwise import cosine_similarity

cluster_transposed = cluster_counts.drop(columns=['cluster_label']).T
similarity_matrix = cosine_similarity(cluster_transposed)
similarity_df = pd.DataFrame(similarity_matrix, index=cluster_transposed.index, columns=cluster_transposed.index)

similarity_df

,cosmos,kava,injective,axelar,osmosis,terra,iris,dydx,secret,akash,stargaze,gravity-bridge,sentinel,juno
cosmos,1.000000,0.459014,0.410356,0.639126,0.779695,0.325504,0.305956,0.688809,0.228836,0.626002,0.408905,0.305590,0.366598,0.493270
kava,0.459014,1.000000,0.341574,0.601772,0.564018,0.291700,0.277375,0.491494,0.240310,0.675746,0.424267,0.966624,0.610660,0.444691
injective,0.410356,0.341574,1.000000,0.506277,0.503704,0.951061,0.211847,0.628307,0.226342,0.419863,0.314331,0.169453,0.334798,0.309194
axelar,0.639126,0.601772,0.506277,1.000000,0.852211,0.470186,0.314872,0.660867,0.320763,0.702740,0.434055,0.498853,0.462534,0.439708
osmosis,0.779695,0.564018,0.503704,0.852211,1.000000,0.350586,0.288506,0.812447,0.310406,0.768804,0.612207,0.377538,0.570176,0.658793
terra,0.325504,0.291700,0.951061,0.470186,0.350586,1.000000,0.190671,0.504570,0.350867,0.323164,0.257888,0.171329,0.347811,0.227785
iris,0.305956,0.277375,0.211847,0.314872,0.288506,0.190671,1.000000,0.722703,0.166614,0.793703,0.184058,0.232116,0.318613,0.198153
dydx,0.688809,0.491494,0.628307,0.660867,0.812447,0.504570,0.722703,1.000000,0.366383,0.925705,0.525888,0.302460,0.616922,0.564808
secret,0.228836,0.240310,0.226342,0.320763,0.310406,0.350867,0.166614,0.366383,1.000000,0.323394,0.213577,0.180070,0.851634,0.184490
akash,0.626002,0.675746,0.419863,0.702740,0.768804,0.323164,0.793703,0.925705,0.323394,1.000000,0.551870,0.556887,0.643195,0.583070


In [16]:
similar_dict = {}
similar_list = []
i = 0
for ch in chain_list:
    similar_chains = list(similarity_df[similarity_df[ch] > 0.7].index)
    similar_chains = [c for c in similar_chains if c != ch]
    similar_dict[ch] = similar_chains
    similar_list.append(similar_chains)

similar_dict

{'cosmos': ['osmosis'],
 'kava': ['gravity-bridge'],
 'injective': ['terra'],
 'axelar': ['osmosis', 'akash'],
 'osmosis': ['cosmos', 'axelar', 'dydx', 'akash'],
 'terra': ['injective'],
 'iris': ['dydx', 'akash'],
 'dydx': ['osmosis', 'iris', 'akash'],
 'secret': ['sentinel'],
 'akash': ['axelar', 'osmosis', 'iris', 'dydx'],
 'stargaze': ['juno'],
 'gravity-bridge': ['kava'],
 'sentinel': ['secret'],
 'juno': ['stargaze']}

In [17]:
chain['similar_chains'] = chain['chain'].map(similar_dict)
chain

,chain,x-axis,y-axis,radius,proposal,proportion,validator_num,proposal_num,cluster_num,similar_chains
0,cosmos,-3.932109,-3.136361,16.389515,"{'23.12-24.02': 25, '24.03...","{0: 0.017793594306049824, ...",281.0,249.0,8.0,[osmosis]
1,kava,-0.472359,1.235081,14.392602,"{'23.12-24.02': 7, '24.03-...","{0: 0.016, 1: 0.064, 2: 0....",125.0,188.0,8.0,[gravity-bridge]
2,injective,-2.156337,-5.232898,14.819475,"{'23.12-24.02': 49, '24.03...","{0: 0.0392156862745098, 1:...",102.0,406.0,8.0,[terra]
3,axelar,-3.018877,-1.306728,13.776378,"{'23.12-24.02': 5, '24.03-...","{0: 0.012048192771084338, ...",83.0,123.0,8.0,"[osmosis, akash]"
4,osmosis,-5.526520,-1.757991,16.635813,"{'23.12-24.02': 50, '24.03...","{0: 0.019305019305019305, ...",259.0,791.0,8.0,"[cosmos, axelar, dydx, akash]"
5,terra,-0.076367,-7.691177,17.385445,"{'23.12-24.02': 6, '24.03-...","{0: 0.12987012987012986, 1...",154.0,103.0,8.0,[injective]
6,iris,3.950026,2.738528,16.020174,"{'23.12-24.02': 20, '24.03...","{0: 0.024793388429752067, ...",121.0,85.0,8.0,"[dydx, akash]"
7,dydx,-2.093366,-1.113769,14.363712,"{'23.12-24.02': 24, '24.03...","{0: 0.060240963855421686, ...",83.0,175.0,8.0,"[osmosis, iris, akash]"
8,secret,4.117796,-3.661395,15.788137,"{'23.12-24.02': 8, '24.03-...","{0: 0.5966386554621849, 1:...",119.0,229.0,8.0,[sentinel]
9,akash,-0.494838,0.632218,12.711289,"{'23.12-24.02': 9, '24.03-...","{0: 0.03225806451612903, 1...",155.0,95.0,8.0,"[axelar, osmosis, iris, dydx]"


# 6. 체인에 속한 검증인 저장

In [28]:
votes = pd.read_csv('./complete_data/validator_votes.csv').drop(columns='Unnamed: 0')

chain_list = [
    'cosmos', 'kava', 'injective', 'axelar', 'osmosis', 'terra',
    'iris', 'dydx', 'secret', 'akash', 'stargaze', 'gravity-bridge',
    'sentinel', 'juno'
]
chain_dfs = {}

for chain_name in chain_list:
    chain_df = votes[votes[chain_name] == 1]
    chain_df = chain_df[['voter']].reset_index(drop=True)    
    chain_dfs[chain_name] = chain_df

chain['validators'] = chain['chain'].apply(lambda x: chain_dfs[x]['voter'].tolist() if x in chain_dfs else [])
chain

/var/folders/nc/dhlk15l910g4hrbcq92y1cy40000gn/T/ipykernel_3603/2922285866.py:1: DtypeWarning: Columns (2,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,61,63,64,65,67,68,69,70,71,72,73,74,75,76,77,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,

,chain,x-axis,y-axis,radius,proposal,proportion,validator_num,proposal_num,cluster_num,similar_chains,validators
0,cosmos,-3.932109,-3.136361,16.389515,"{'23.12-24.02': 25, '24.03...","{0: 0.017793594306049824, ...",281.0,249.0,8.0,[osmosis],"[Chainflow, Imperatorco, S..."
1,kava,-0.472359,1.235081,14.392602,"{'23.12-24.02': 7, '24.03-...","{0: 0.016, 1: 0.064, 2: 0....",125.0,188.0,8.0,[gravity-bridge],[ AutoStake Slash Protect...
2,injective,-2.156337,-5.232898,14.819475,"{'23.12-24.02': 49, '24.03...","{0: 0.0392156862745098, 1:...",102.0,406.0,8.0,[terra],"[Imperatorco, AutoStake ..."
3,axelar,-3.018877,-1.306728,13.776378,"{'23.12-24.02': 5, '24.03-...","{0: 0.012048192771084338, ...",83.0,123.0,8.0,"[osmosis, akash]","[Imperatorco, AutoStake ..."
4,osmosis,-5.526520,-1.757991,16.635813,"{'23.12-24.02': 50, '24.03...","{0: 0.019305019305019305, ...",259.0,791.0,8.0,"[cosmos, axelar, dydx, akash]","[Inactive Mercury, Chain..."
5,terra,-0.076367,-7.691177,17.385445,"{'23.12-24.02': 6, '24.03-...","{0: 0.12987012987012986, 1...",154.0,103.0,8.0,[injective],"[Imperatorco, AutoStake ..."
6,iris,3.950026,2.738528,16.020174,"{'23.12-24.02': 20, '24.03...","{0: 0.024793388429752067, ...",121.0,85.0,8.0,"[dydx, akash]","[Imperatorco, SpacePotato,..."
7,dydx,-2.093366,-1.113769,14.363712,"{'23.12-24.02': 24, '24.03...","{0: 0.060240963855421686, ...",83.0,175.0,8.0,"[osmosis, iris, akash]","[Imperatorco, StakeWithUs,..."
8,secret,4.117796,-3.661395,15.788137,"{'23.12-24.02': 8, '24.03-...","{0: 0.5966386554621849, 1:...",119.0,229.0,8.0,[sentinel],[ AutoStake Slash Protect...
9,akash,-0.494838,0.632218,12.711289,"{'23.12-24.02': 9, '24.03-...","{0: 0.03225806451612903, 1...",155.0,95.0,8.0,"[axelar, osmosis, iris, dydx]","[Chainflow, Imperatorco, ..."


# 7. 결과 저장

In [29]:
chain.to_csv('./complete_data/chain_data.csv')
chain.to_json('./complete_data/chain_data.json', orient='records')